<a href="https://colab.research.google.com/github/quadribello/Hamoye/blob/master/quadri_bello_quiz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# importing important libraries
import pandas as pd
import numpy as np

In [2]:
from google.colab import files
uploaded = files.upload()

Saving Data_for_UCI_named.csv to Data_for_UCI_named (1).csv


In [3]:
# loading dataset
df = pd.read_csv('Data_for_UCI_named.csv')
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


In [11]:
# shape of the dataset
df.shape

(10000, 14)

In [12]:
# checking for missing values
df.isna().sum()

tau1     0
tau2     0
tau3     0
tau4     0
p1       0
p2       0
p3       0
p4       0
g1       0
g2       0
g3       0
g4       0
stab     0
stabf    0
dtype: int64

In [4]:
df.stabf.value_counts()

unstable    6380
stable      3620
Name: stabf, dtype: int64

In [5]:
# Data preprocessing
X = df.drop(columns=['stab','stabf'])
y = df['stabf']

In [6]:
# Train Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state = 1)

In [14]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_train_df = scaler.fit_transform(X_train)
scaled_train_df = pd.DataFrame(scaled_train_df,columns=X_train.columns)
scaled_test_df = scaler.transform(X_test)
scaled_test_df = pd.DataFrame(scaled_test_df,columns=X_test.columns)

In [17]:
# QUESTION 1

TP = 355
FP = 1480
FN = 45
TN = 120

precision = 355/(355 + 1480)
recall = 355/(355 + 45)

F1 = 2 * (precision*recall)/(precision+recall)
F1

0.3176733780760626

In [18]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(random_state=1)
classifier.fit(scaled_train_df, y_train)
#prediction
y_pred = classifier.predict(scaled_test_df)

In [19]:
# Evaluating the Algorithm
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(accuracy_score(y_test, y_pred))

0.929


In [20]:
from xgboost import XGBClassifier
clf_xgb = XGBClassifier(random_state=1)
clf_xgb.fit(scaled_train_df, y_train)
clf_xgb_pred = clf_xgb.predict(scaled_test_df)
print(accuracy_score(y_test, clf_xgb_pred))

0.9195


In [21]:
import lightgbm as lgb
clf_lgb = lgb.LGBMClassifier(random_state=1)
clf_lgb.fit(scaled_train_df, y_train)
clf_lgb_pred = clf_lgb.predict(scaled_test_df)
print(accuracy_score(y_test, clf_lgb_pred))

0.9375


In [22]:
from sklearn.ensemble import ExtraTreesClassifier
E_classifier = ExtraTreesClassifier(random_state=1)
E_classifier.fit(scaled_train_df, y_train)
#prediction
yE_pred = E_classifier.predict(scaled_test_df)
df_check_E = pd.DataFrame({'Actual':y_test, 'Predicted':y_pred})
df_check_E

,Actual,Predicted
9953,unstable,unstable
3850,unstable,unstable
4962,stable,stable
3886,stable,stable
5437,unstable,unstable
...,...,...
3919,stable,stable
162,stable,stable
7903,stable,stable
2242,unstable,stable


In [27]:
print(accuracy_score(y_test, yE_pred))

0.928


In [23]:
# Hyperparameters
n_estimators = [50, 100, 300, 500, 1000]

min_samples_split = [2, 3, 5, 7, 9]

min_samples_leaf = [1, 2, 4, 6, 8]

max_features = ['auto', 'sqrt', 'log2', None] 

hyperparameter_grid = {'n_estimators': n_estimators,

                       'min_samples_leaf': min_samples_leaf,

                       'min_samples_split': min_samples_split,

                       'max_features': max_features}

In [24]:
# Instantiate the RandomizedSearchCV object: tree_cv
from sklearn.model_selection import RandomizedSearchCV
RandomizedSearchCV = RandomizedSearchCV(estimator = E_classifier, cv=5, n_iter=10, param_distributions=hyperparameter_grid, scoring='accuracy', n_jobs=-1, verbose =1, random_state=1)
RandomizedSearchCV = RandomizedSearchCV.fit(scaled_train_df, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  1.5min finished


In [25]:
print("Best score is {}".format(RandomizedSearchCV.best_score_))
RandomizedSearchCV.best_params_

Best score is 0.9241249999999999


{'max_features': None,
 'min_samples_leaf': 8,
 'min_samples_split': 2,
 'n_estimators': 1000}

In [30]:
best_E_classifier = ExtraTreesClassifier(random_state=1, n_estimators=1000, min_samples_split=2, min_samples_leaf=8, max_features=None)
best_E_classifier.fit(scaled_train_df,y_train)
best_E_classifier_pred = best_E_classifier.predict(scaled_test_df)

In [31]:
print(accuracy_score(y_test, best_E_classifier_pred))

0.927


In [33]:
# Question 20
features_importance = RandomizedSearchCV.best_estimator_.feature_importances_
cols = X.columns
most_important_feature = features_importance.max()
cols[features_importance == most_important_feature][0]

'tau2'

In [34]:
least_important_feature = features_importance.min()
cols[features_importance == least_important_feature][0]

'p1'